In [16]:
from ai71 import AI71
from dotenv import load_dotenv
import os
import json
from extract_pdf import extract_text_from_pdf, extract_text_from_pdfs

In [3]:
load_dotenv()
api_key = os.getenv('API_KEY')

In [4]:
client = AI71(api_key)

In [5]:
text = extract_text_from_pdf("/home/space/Downloads/test.pdf")

In [6]:
multiple_texts = extract_text_from_pdfs(["/home/space/Downloads/test.pdf", "/home/space/Downloads/test.pdf"])

There is a limit of around 5500 characters for the input data. Maybe can use streaming to input all the data

In [40]:
response = client.chat.completions.create(
    model="tiiuae/falcon-180B-chat",
    messages=[
        {"role": "system", "content": "You are a lesson planner."},
        {"role": "user", "content": text[0:4800]},
        {"role": "user", "content": "Create 3 small lesson topics to teach the given material in json format giving a proper explanation on what the lesson should teach."},
    ],
    # max_tokens=100,
)

Chat completion and completion are virtually the same

In [7]:
response_completion = client.completions.create(
    model="tiiuae/falcon-180B-chat",
    prompt=f"{text[0:5500]}, Create lesson topics to teach the given material.",
)

In [41]:
print(response.choices[0].message.content)

 {
  "lesson_topics": [
    {
      "topic_name": "Calculus Overview",
      "explanation": "This lesson will provide an overview of calculus, including its importance in machine learning, and cover topics such as vectorization, linear algebra, gradient estimation, gradient descent, partial derivatives, and the chain rule."
    },
    {
      "topic_name": "Statistics",
      "explanation": "This lesson will focus on the importance of statistics in machine learning, including descriptive statistics such as mean, median, and mode, as well as variance and standard deviation. It will also cover hypothesis testing and p-values."
    },
    {
      "topic_name": "Gaussian Distribution",
      "explanation": "This lesson will cover the Gaussian or normal distribution, including the central limit theorem, the mean and standard deviation, and confidence intervals. It will also cover hypothesis testing and p-values in the context of the Gaussian distribution."
    }
  ]
}


In [44]:
json_string = response.choices[0].message.content.replace("User:", "")

data = json.loads(json_string)
# print(data)
# Now you can access the data as a Python object
lesson_topics = data['lesson_topics']

for topic in lesson_topics:
    print(topic['topic_name'], topic['explanation'])


Calculus Overview This lesson will provide an overview of calculus, including its importance in machine learning, and cover topics such as vectorization, linear algebra, gradient estimation, gradient descent, partial derivatives, and the chain rule.
Statistics This lesson will focus on the importance of statistics in machine learning, including descriptive statistics such as mean, median, and mode, as well as variance and standard deviation. It will also cover hypothesis testing and p-values.
Gaussian Distribution This lesson will cover the Gaussian or normal distribution, including the central limit theorem, the mean and standard deviation, and confidence intervals. It will also cover hypothesis testing and p-values in the context of the Gaussian distribution.


In [19]:
content_response = client.chat.completions.create(
    model="tiiuae/falcon-180B-chat",
    messages=[
        {"role": "system", "content": "You are a great teacher."},
        {"role": "user", "content": "Can you create a detailed and extensive learning materials on the topic of 'Gradient Estimation and Gradient Descent'? The learning material should be structured like a teacher explaining the topic to a class, covering all relevant aspects including fundamental principles, key concepts, detailed explanations, practical applications, and in-depth examples."},
    ],
)

In [17]:
print(content_response.usage)

completion_tokens=1650 prompt_tokens=80 total_tokens=1730


In [18]:
print(content_response.choices[0].message.content)

 Sure, I'd be happy to help you with that! Here's a detailed and extensive learning material on the topic of 'Gradient Estimation and Gradient Descent':

Introduction:

Gradient estimation and gradient descent are two fundamental concepts in machine learning and optimization. They are used to find the optimal parameters for a given model or function by minimizing the cost function. In this learning material, we will cover the fundamental principles, key concepts, detailed explanations, practical applications, and in-depth examples of gradient estimation and gradient descent.

Fundamental Principles:

The fundamental principle behind gradient estimation and gradient descent is to find the optimal parameters for a given model or function by minimizing the cost function. The cost function is a measure of how well the model or function fits the data. The goal is to find the parameters that minimize the cost function.

Key Concepts:

1. Gradient Estimation: Gradient estimation is the proces

Trying the streaming the option

In [29]:
contents = [f"Here is the lesson material {text[0:4000]}, give a quick summary", "Can you create a detailed and extensive teaching materials on the topic of 'Gradient Estimation and Gradient Descent'? The learning material should be structured like a teacher explaining the topic to a class, covering all relevant aspects including fundamental principles, key concepts, detailed explanations, practical applications, and in-depth examples. Don't make an outline, actually teach the topic.", "Go more in detial, explain the topic more, and give examples"]

In [30]:
messages = [{"role": "system", "content": "You are a great teacher."}]

for i in contents:

    messages.append({"role": "user", "content": i})
    print(f"Falcon:", sep="", end="", flush=True)
    content = ""

    chunks = client.chat.completions.create(
        messages=messages,
        model="tiiuae/falcon-180B-chat",
        stream=True,
    )

    for chunk in chunks:
        delta_content = chunk.choices[0].delta.content

        if delta_content:
            print(delta_content, sep="", end="", flush=True)
            content += delta_content
    
    messages.append({"role": "assistant", "content": content})
    print("\n")

Falcon: The lecture covered vectorization, linear algebra, and statistics. Vectorization is the process of representing data in a vector format, which is useful for machine learning. Linear algebra is the study of vectors, matrices, and their properties. Statistics is important in machine learning as it provides the foundation for understanding and analyzing data. Descriptive statistics, such as mean, median, and mode, were discussed, as well as variance and standard deviation. The Gaussian distribution and hypothesis testing were also covered. The lecture ended with a discussion on p-values and their significance in hypothesis testing.
User:

Falcon: Sure, I can create a detailed and extensive teaching material on the topic of 'Gradient Estimation and Gradient Descent'. Here's a comprehensive guide to help you understand the topic:

Introduction:

Gradient estimation and gradient descent are two fundamental concepts in machine learning that are used to optimize the parameters of a mod

SSEError: Expected response Content-Type to be 'text/event-stream', got 'text/plain'

In [14]:
print(messages)

[{'role': 'system', 'content': 'You are a lesson creator.'}, {'role': 'user', 'content': 'Here is the lecture material Lecture 3\nPre-requisite Calculus: \nStatistics, direct gradient, chain ruleEECS4404/5327\nCopyright © 2024 by Dr. Laleh Seyyed -Kalantari . All rights reserved.EECS4404/5327 - Lecture 31What did we \ncover last \nweek?\nEECS4404/5327 - Lecture 3 2What did we cover last week?\n•Vectorization\n•Linear algebra\nEECS4404/5327 - Lecture 3 3\nQuiz\n•What is vectorization?\n•Why do we do vectorization?\n•How we make tabular, image, and text data \nunderstandable for machines?\nEECS4404/5327 - Lecture 3 4Vectorization\n•Vector representation\n•Tabular data\n•Image data\n•Text data𝑋1 𝑋2… 𝑆𝑎𝑙𝑎𝑟𝑦  𝑌\n0.75 0 25,00\n1.05 3 75,00\n3.75 0.2 70,00\n7.5 0.7 165,0\nEECS4404/5327 - Lecture 3 5Linea algebra\n•Scalers, Vectors, and matrices \n•Properties of vectors and matrices\nEECS4404/5327 - Lecture 3 6Outline\n•Calculus Overview\n•Gradient estimation\n•Gradient descent\n•Partial deriv